In [1]:
#|default_exp classification.baseline.baseline

In [2]:
#|export
from fastai.vision.all import *
from torch.utils.data import TensorDataset

# Baseline
> Quick baselines to compare your model results to.

In [3]:
#|export
class FastaiMetricWrapper(GetAttr):
    _default = 'metric'
    def __init__(self, metric):
        self.metric = metric
        
    def accumulate(self, pred, yb):
        fake_learner = namedtuple('FakeLearner', 'pred yb y to_detach')(pred, (yb,), yb, to_detach)
        self.metric.accumulate(fake_learner)

In [4]:
#|export
def calculate_metrics(dl, metrics, probs_func, n_classes=None):
    "Calculate metrics for a baseline method `probs_func`"
    n_classes = n_classes or get_c(dl)
    if n_classes == 0:
        raise ValueError("Couldn't automatically infer number of classes from dataloader. Please explicitly specify it with `n_classes`")
        
    metrics = L(metrics).map(compose(mk_metric, FastaiMetricWrapper))
    for metric in metrics: metric.reset()

    for x,y in dl:
        pred = probs_func(x, y, n_classes)
        for metric in metrics: metric.accumulate(pred, y)

    return {metric.name: metric.value for metric in metrics}

In [5]:
X = torch.randn(10, 1)
Y = tensor([0, 1, 3, 2, 0, 1, 1, 0, 0, 2])

dl = TfmdDL(TensorDataset(X, Y), bs=4)

x, y = first(dl)

In [6]:
test_fail(lambda: calculate_metrics(dl, None, None), contains='classes')

In [7]:
metrics =  [accuracy, error_rate, F1Score(average='macro')]

In [8]:
#|export
def fixed_probs(x, y, n_classes, class_id):
    pred = torch.zeros(find_bs(y), n_classes).to(y.device)
    pred[:, class_id] = 1
    return pred

In [9]:
pred = fixed_probs(x, y, 4, 1)

test_eq(pred, tensor([[0., 1., 0., 0.],
                      [0., 1., 0., 0.],
                      [0., 1., 0., 0.],
                      [0., 1., 0., 0.]]))

In [10]:
res = calculate_metrics(dl, metrics, partial(fixed_probs, class_id=2), 3)
test_eq(res, {'accuracy': 0.2000, 'error_rate': 0.8000, 'f1_score': 0.08333333333333334})

In [11]:
#|export
def random_probs(x, y, n_classes):
    pred = torch.randn(find_bs(y), n_classes).to(y.device)
    pred = torch.nn.functional.softmax(pred, dim=-1)
    return pred

In [12]:
random_probs(x, y, 4)

tensor([[0.1449, 0.2503, 0.1713, 0.4335],
        [0.3958, 0.2078, 0.1248, 0.2716],
        [0.3533, 0.3805, 0.1470, 0.1192],
        [0.1083, 0.5671, 0.0373, 0.2873]])

In [13]:
calculate_metrics(dl, metrics, random_probs, 3)

{'accuracy': TensorBase(0.3000),
 'error_rate': TensorBase(0.7000),
 'f1_score': 0.2361111111111111}

In [14]:
# Doesn't make much sense? Because when probs are converted to preds with `argmax` the first one will always be picked
# def uniform_probs(x, y, n_classes):
#     pred = torch.ones(find_bs(y), n_classes).to(y.device)
#     pred /= n_classes
#     return pred

In [15]:
# pred = uniform_probs(x, y, 4)
# test_eq(pred, tensor([[0.2500, 0.2500, 0.2500, 0.2500],
#                       [0.2500, 0.2500, 0.2500, 0.2500],
#                       [0.2500, 0.2500, 0.2500, 0.2500],
#                       [0.2500, 0.2500, 0.2500, 0.2500]]))

In [16]:
#|hide
from nbdev import nbdev_export
nbdev_export()